In [43]:
# Loading necessary libraries
import pandas as pd
import sqlite3
from datetime import datetime, timedelta

In [44]:
# Reading the data
def run_etl(file_path, db_path="retail_dw.db"):
    """
    Full ETL process:
    1. Extract data from Excel
    2. Transform (clean, enrich, filter)
    3. Load into SQLite database
    Returns: Dictionary log with row counts at each stage
    """

In [45]:
# Extraction 
# Read only the columns we actually need for the warehouse
file_path = "Online Retail.xlsx"
use_cols = [
    "InvoiceNo", "StockCode", "Description",
    "Quantity", "InvoiceDate", "UnitPrice",
    "CustomerID", "Country"
]
df = pd.read_excel(file_path, usecols=use_cols)

In [46]:
# Data Cleaning
# Drop rows that have missing key values:
# CustomerID, InvoiceDate, Quantity, UnitPrice must all be present
df.dropna(subset=["CustomerID", "InvoiceDate", "Quantity", "UnitPrice"], inplace=True)

# Convert InvoiceDate from string/object to datetime format
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")

# Drop rows that failed the datetime conversion (errors='coerce' creates NaT for invalid dates)
df.dropna(subset=["InvoiceDate"], inplace=True)

log = {}
# Log the number of rows after extraction
log["extracted_rows"] = len(df)

In [47]:
log # This will output the log as cell result

{'extracted_rows': 406829}

In [48]:
# Remove outliers:
# - Quantity should be >= 0
# - UnitPrice should be > 0
df = df[(df["Quantity"] >= 0) & (df["UnitPrice"] > 0)]
log["after_outlier_removal"] = len(df)

# Create a new column "TotalSales" = Quantity × UnitPrice
df["TotalSales"] = df["Quantity"] * df["UnitPrice"]

# Create the Customer Dimension:
# - TotalPurchases = sum of TotalSales for each customer
# - Country = country of first transaction (assumes it doesn't change)
customer_summary = df.groupby("CustomerID").agg(
    TotalPurchases=("TotalSales", "sum"),
    Country=("Country", "first")
).reset_index()

# Use all available data
df_last_year = df.copy() 

# Create the Time Dimension table:
# - Each unique date in sales
# - Include year, month, and day columns
time_dim = pd.DataFrame({
    "InvoiceDate": pd.to_datetime(df_last_year["InvoiceDate"].dt.date.unique())
})
time_dim["Year"] = time_dim["InvoiceDate"].dt.year
time_dim["Month"] = time_dim["InvoiceDate"].dt.month
time_dim["Day"] = time_dim["InvoiceDate"].dt.day

In [49]:
log # This will output the log as cell result

{'extracted_rows': 406829, 'after_outlier_removal': 397884}

In [50]:
# Define the database path
db_path = "retail_dw.db"

# Connect to SQLite (creates DB file if not exists)
conn = sqlite3.connect(db_path)

# Load the Fact table (SalesFact) - contains transactional data
df_last_year.to_sql("SalesFact", conn, if_exists="replace", index=False)

# Load the Customer Dimension table
customer_summary.to_sql("CustomerDim", conn, if_exists="replace", index=False)

# Load the Time Dimension table
time_dim.to_sql("TimeDim", conn, if_exists="replace", index=False)

# Commit changes & close connection
conn.commit()
conn.close()

log.update({
    'sales_records': len(df_last_year),
    'customer_records': len(customer_summary),
    'time_periods': len(time_dim),
    'status': 'success'
})

In [51]:
log  # This will output the log as cell result

{'extracted_rows': 406829,
 'after_outlier_removal': 397884,
 'sales_records': 397884,
 'customer_records': 4338,
 'time_periods': 305,
 'status': 'success'}

In [52]:
if __name__ == "__main__":
    file_path = "Online Retail.xlsx"
    # Run ETL and get log info
    log_info = run_etl(file_path)
    print("ETL Log:", log_info)

    # Example Queries after Load
   
    conn = sqlite3.connect("retail_dw.db")

    # Query 1: Total sales by country
    q1 = pd.read_sql("""
        SELECT Country, SUM(TotalSales) AS TotalSales
        FROM SalesFact
        GROUP BY Country
        ORDER BY TotalSales DESC
    """, conn)
    print("\nTotal Sales by Country:\n", q1)

    # Query 2: Top 10 customers by purchases
    q2 = pd.read_sql("""
        SELECT * FROM CustomerDim
        ORDER BY TotalPurchases DESC
        LIMIT 10
    """, conn)
    print("\nTop 10 Customers:\n", q2)

    # Query 3: Monthly sales trend (join Fact & Time dimensions)
    q3 = pd.read_sql("""
        SELECT Year, Month, SUM(TotalSales) AS MonthlySales
        FROM TimeDim
        JOIN SalesFact
        ON date(SalesFact.InvoiceDate) = date(TimeDim.InvoiceDate)
        GROUP BY Year, Month
        ORDER BY Year, Month
    """, conn)
    print("\nMonthly Sales Trend:\n", q3)

    conn.close()

ETL Log: None

Total Sales by Country:
                  Country    TotalSales
0         United Kingdom  7.308392e+06
1            Netherlands  2.854463e+05
2                   EIRE  2.655459e+05
3                Germany  2.288671e+05
4                 France  2.090240e+05
5              Australia  1.385213e+05
6                  Spain  6.157711e+04
7            Switzerland  5.644395e+04
8                Belgium  4.119634e+04
9                 Sweden  3.837833e+04
10                 Japan  3.741637e+04
11                Norway  3.616544e+04
12              Portugal  3.343989e+04
13               Finland  2.254608e+04
14             Singapore  2.127929e+04
15       Channel Islands  2.045044e+04
16               Denmark  1.895534e+04
17                 Italy  1.748324e+04
18                Cyprus  1.359038e+04
19               Austria  1.019868e+04
20                Poland  7.334650e+03
21                Israel  7.221690e+03
22                Greece  4.760520e+03
23               Iceland